In [ ]:
#the filter we use to filter some noise in the document
import re
def change_filter(doc):
    txt2 = re.sub(r',|\.|\?|<br|/>|\:|;|\(|\)',"",doc)
    txt3 = re.sub(r'\'ll'," will",txt2)
    txt4 = re.sub(r'\'m'," am",txt3)
    txt5 = re.sub(r'\'s'," is",txt4)
    lowered_tokens = txt5.lower()
    return lowered_tokens

In [ ]:
#the vectorizer to vectorize our text samples (unigram model)
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1) 
X = vectorizer.fit_transform(f)
analyze = vectorizer.build_analyzer()

In [ ]:
#the vectorizer to vectorize our text samples (bigram model)
from sklearn.feature_extraction.text import CountVectorizer
bigram_vectorizer = CountVectorizer(ngram_range=(2,2),token_pattern=r'\b\w+\b', min_df=1)
analyze = bigram_vectorizer.build_analyzer()

In [ ]:
#the vectorizer to vectorize our text samples (unigram+bigram model)
from sklearn.feature_extraction.text import CountVectorizer
bigram_vectorizer = CountVectorizer(ngram_range=(2,2),token_pattern=r'\b\w+\b', min_df=1)
analyze = bigram_vectorizer.build_analyzer()

In [ ]:
#generate vectors for document to train with SVM
import numpy as np
# posPath = "/Users/chenjiannan/Documents/UMass/2016Fall/CS585-INLP/project/restaurants/large_movie_review_dataset/test/pos"
# negPath = "/Users/chenjiannan/Documents/UMass/2016Fall/CS585-INLP/project/restaurants/large_movie_review_dataset/test/neg"
os.chdir(posPath)
filenames = os.listdir(posPath)
allFiles = []
for i in filenames:
    if os.path.splitext(i)[1] == '.txt':
        allFiles.append(i)
print(len(allFiles))
flag = True
data = np.array([2,3,1,0])
for f in allFiles:
    if flag == True:
        with open(os.path.join(posPath,f),'r',encoding='latin1') as doc:
            words = []
            content = better_Tokenizer(doc.read())
            words.append(content)
            data = vectorizer.transform(words).toarray()
            flag = False
    else:
        with open(os.path.join(posPath,f),'r',encoding='latin1') as doc:
            words = []
            content = better_Tokenizer(doc.read())
            words.append(content)
            data = np.row_stack((data, vectorizer.transform(words).toarray()))

In [ ]:
#used for SVM training and testing 
from liblinearutil import *
y, x = svm_read_problem('testtrainnew.txt')
prob = problem(y, x)
param = parameter('-s 3 -c 0.000004')
m = train(prob, param)
CV_ACC = train(y, x, '-v 3')
best_C, best_rate = train(y, x, '-C -s 0')
y1, x1 = svm_read_problem('testtestnew.txt')
p_labs, p_acc, p_vals = predict(y1, x1, m, '-q')
print(p_acc)

In [ ]:
#read from inspection records and restaurant records
import gzip
import os

f = open(os.path.join(os.getcwd(),"restaurant_inspections.csv"),"r").readlines()
file1 = open('r_inspections3.csv','w')
for line in f:
    b = line.split(';')
    file1.write(b[0]+","+b[1]+","+b[2]+","+b[3]+","+b[4]+","+b[5]+","+b[6]+","+b[7]+","+b[8]+","+b[9]+","+b[10]+","+b[11]+","+b[12]+","+b[13]+"\n")

In [ ]:
#address matching 
import re
import os
from streetaddress import StreetAddressFormatter, StreetAddressParser
from collections import defaultdict

addr_parser = StreetAddressParser()
f1 = open(os.path.join(os.getcwd(),"restaurant_establishments.csv"),"r").readlines()
file2 = open('r_est2.txt','w')
tSet = set()
tDict = defaultdict(str)
print("len1",len(f1))
pattern3 = re.compile(r'^[snew]\s')
for line in f1[1:]:
    num = re.sub(r';', " ", line)
    b = line.split(";")
    facility_id = b[1]
    b2 = re.sub(r"\n","",b[5])
    b2 = re.sub(r"\'|\*|\.|\,|\n","",b2)
    b2 = b2.lower()
    addr = defaultdict(str)
    addr = addr_parser.parse(b2)
    street_name = str(addr['street_name'])
    result = re.match(pattern3, street_name)
    house = str(addr['house'])
    street_type = str(addr['street_type'])
    if result:
        street_name = addr['street_name'][2:]
    if house =='None':
        house = ''
    else:
        house = house+" "
    if street_type=='None':
        street_type = ''
    else:
        street_type = " "+street_type
    address = house+street_name+street_type
    if address not in tDict.keys():
        tDict[address] = facility_id
        file2.write(address+";"+facility_id+"\n")
    tSet.add(address)
print("len1",len(tSet))
print(tDict)
print("len4",len(tDict))

import json
f3 = open(os.path.join(os.getcwd(),"vegas_business.txt"),"r").readlines()
file3 = open('r_business.txt','w')
pattern1 = re.compile(r'\d+.+')
pattern2 = re.compile(r'.+\n(\d+.+)\s+\S+')
count = 1
f4=open('result.txt','w')
print(len(f3))
for line in f3:
    decode = json.loads(line)
    st = decode['full_address']
    name = decode['name']
    business_id = decode['business_id']
    result = re.match(pattern1, st)
    if result:
        print(result.group(),"---",count)
        f4.write(name+";"+result.group()+";"+business_id+'\n')
        count=count+1
    else:
        result2 = re.match(pattern2, st)
        if result2:
            print(result2.group(1),"---",count)
            f4.write(name+";"+result2.group(1)+";"+business_id+'\n')
            count=count+1
        else:
            print(st)
f4.close()

from streetaddress import StreetAddressFormatter, StreetAddressParser
f5 = open(os.path.join(os.getcwd(),"result.txt"),"r").readlines()
file4 = open('r_business1.txt','w')
print("len3",len(f5))
pattern4 = re.compile(r'^[snew]\s')
count = 0
oSet = set()
newSet = set()
for line in f5:
    s = re.sub(r"\n","",line)
    segments = s.split(';')
    s1 = segments[0]
    s2 = segments[1]
    s1 = s1.lower()
    s2 = s2.lower()
    s1 = re.sub(r"\'|\*|\.|\,|\n","",s1)
    s2 = re.sub(r"\'|\*|\.|\,|\n","",s2)
    if s2 in oSet:
        print(s2)
    oSet.add(s2)
    addr = addr_parser.parse(s2)
    street_name = str(addr['street_name'])
    result = re.match(pattern3, street_name)
    house = str(addr['house'])
    street_type = str(addr['street_type'])
    if result:
        street_name = addr['street_name'][2:]
    if house =='None':
        house = ''
    else:
        house = house+" "
    if street_type=='None':
        street_type = ''
    else:
        street_type = " "+street_type
    address = house+street_name+street_type
    newSet.add(address)
    if address in tSet:
        print(s1,",",address,",",segments[2])
        tSet.remove(address)
        file4.write(s1+";"+address+";"+segments[2]+"\n")
        count=count+1
print("len4",count)
print("len5",len(newSet))
print("len5",len(oSet))

In [ ]:
#facility id to business id
import os
import json
f6 =open('r_est2.txt','r').readlines()
f5 = open('r_business1.txt','r').readlines()
# file10 = open('fidTobid','w')
fname6 = []
fname5 = []
fidTobid = {}
bidTofid = {}
for line in f6:
    b = line.split(";")
    for line1 in f5:
        b1 = line1.split(";")
        if b[0]==b1[1]:
            fidTobid[b[1][:-1]] = b1[2][:-1]
            bidTofid[b1[2][:-1]] = b[1][:-1]
# file10.write(str(fidTobid))
print(fidTobid)
print(bidTofid)

In [ ]:
#generate inspection list for each facility
f5 = open('r_business1.txt','r').readlines()
f6 =open('r_est2.txt','r').readlines()
time_span = defaultdict(list)
fname5 = set()
fname6 = defaultdict(str)
for line in f6:
    b = line.split(";")
    fname6[b[0]] = b[1][:-1]
# print(fname6)
for line in f5:
    b = line.split(";")
    inspection_id = fname6[b[1]]
    fname5.add(b[1])
#     print(inspection_id)
    spanList = sorted(getTimeSpan(inspection_id))
#     print(spanList)
    if len(spanList)<3 and len(spanList)>0:
        time_span[inspection_id] = spanList
print(time_span)

In [ ]:
#create samples for both classes from reviews
import json 

def getReviewsNew(ilist, fileLines):
    posDir = 'posNew/'
    negDir = 'negNew/'
    removeList = []
#     f3 = open(os.path.join(os.getcwd(),"vegas_reviews_new.json"),"r").readlines()
    timeStart = ilist[0][3]
    fid = ilist[0][2]
    reviews = ''
    for line in fileLines:
        decode = json.loads(line)
        time = datetime.datetime.strptime(decode['date'],"%Y-%m-%d")
        bid = fidTobid[fid]
        if bid == decode['business_id'] and time<timeStart:
            reviews = reviews+decode['text']
            removeList.append(line)
    if ilist[0][7]>10 and len(reviews)>0:
        fileTemp = open(posDir+fid+str(timeStart)+".txt",'w')
        fileTemp.write(reviews)
        fileTemp.close()
    elif ilist[0][7]<=10 and len(reviews)>0:
        fileTemp = open(negDir+fid+str(timeStart)+".txt",'w')
        fileTemp.write(reviews)
        fileTemp.close()
    for k in range(0,len(ilist)-1):
        seg1 = ilist[k]
        seg2 = ilist[k+1]
        timeStart = seg1[3]
        timeEnd = seg2[3]
        reviews = ''
        for line in fileLines:
            decode = json.loads(line)
            time = datetime.datetime.strptime(decode['date'],"%Y-%m-%d")
            bid = fidTobid[fid]
            if bid == decode['business_id'] and time>timeStart and time<timeEnd:
                reviews = reviews+decode['text']
                removeList.append(line)
        if seg2[7]>10 and len(reviews)>0:
            fileTemp = open(posDir+fid+str(timeEnd)+".txt",'w')
            fileTemp.write(reviews)
            fileTemp.close()
        elif seg2[7]<=10 and len(reviews)>0:
            fileTemp = open(negDir+fid+str(timeEnd)+".txt",'w')
            fileTemp.write(reviews)
            fileTemp.close()
    return removeList

In [ ]:
#get statistical features like average rating, non-positive review count and review count
import json 
from __future__ import division
def getFeatures(ilist, fileLines):
    posDir = 'posNew/'
    negDir = 'negNew/'
    removeList = []
#     f3 = open(os.path.join(os.getcwd(),"vegas_reviews_new.json"),"r").readlines()
    timeStart = ilist[0][3]
    fid = ilist[0][2]
    reviews = ''
    totalRating = 0
    negReviews = 0
    reviewWords = 0
    reviewCount = 0 
    for line in fileLines:
        decode = json.loads(line)
        time = datetime.datetime.strptime(decode['date'],"%Y-%m-%d")
        bid = fidTobid[fid]
        if bid == decode['business_id'] and time<timeStart:
            reviews = reviews+decode['text']
            reviewWords = reviewWords+len(reviews.split())
            rating = float(decode['stars'])
            totalRating = totalRating+rating
            reviewCount=reviewCount+1
            if rating<=3:
                negReviews = negReviews+1
            removeList.append(line)
    if ilist[0][7]>10 and len(reviews)>0:
        fileTemp = open(posDir+fid+str(timeStart)+".txt",'w')
        fileTemp.write("rc:"+str(reviewWords)+" ar:"+str(totalRating/reviewCount)+" nr:"+str(negReviews))
        fileTemp.close()
    elif ilist[0][7]<=10 and len(reviews)>0:
        fileTemp = open(negDir+fid+str(timeStart)+".txt",'w')
        fileTemp.write("rc:"+str(reviewWords)+" ar:"+str(totalRating/reviewCount)+" nr:"+str(negReviews))
        fileTemp.close()
    for k in range(0,len(ilist)-1):
        seg1 = ilist[k]
        seg2 = ilist[k+1]
        timeStart = seg1[3]
        timeEnd = seg2[3]
        reviews = ''
        totalRating = 0
        negReviews = 0
        reviewWords = 0
        reviewCount = 0 
        for line in fileLines:
            decode = json.loads(line)
            time = datetime.datetime.strptime(decode['date'],"%Y-%m-%d")
            bid = fidTobid[fid]
            if bid == decode['business_id'] and time>timeStart and time<timeEnd:
                reviews = reviews+decode['text']
                reviewWords = reviewWords+len(reviews.split())
                rating = float(decode['stars'])
                totalRating = totalRating+rating
                reviewCount=reviewCount+1
                if rating<=3:
                    negReviews = negReviews+1
                removeList.append(line)
        if seg2[7]>10 and len(reviews)>0:
            fileTemp = open(posDir+fid+str(timeEnd)+".txt",'w')
            fileTemp.write("rc:"+str(reviewWords)+" ar:"+str(totalRating/reviewCount)+" nr:"+str(negReviews))
            fileTemp.close()
        elif seg2[7]<=10 and len(reviews)>0:
            fileTemp = open(negDir+fid+str(timeEnd)+".txt",'w')
            fileTemp.write("rc:"+str(reviewWords)+" ar:"+str(totalRating/reviewCount)+" nr:"+str(negReviews))
            fileTemp.close()
    return removeList